In [1]:
import glob
print(glob.glob("/share/*"))

['/share/input.txt', '/share/us-stock-dataset', '/share/Transformers1']


In [2]:
import sys
import os

import torch
from torch import nn as nn
import glob

from t4.timeseries_transformer import TransformerConfig, TransformerRunner

from timeseries.csv_reader import read_and_merge_csv_files

from t3_karpathy.commons.commons import BaseTransformerConfig
from t3_karpathy.token_codec import TokenCodec
from t4.generic_dataloader import GenericDataloader
from timeseries.timeseries_transformer import TimeseriesDataloader



In [3]:
stocks_to_load = [
    "AAPL", "TSLA",
    "A", "GOOG", "AMZN", "PYPL", "NVDA", "AMD",
    "NFLX", "MSFT", "INTC", "CSCO", "ADBE", "CRM", "QCOM", "TXN", "AVGO",
    "INTU", "ORCL", "COST", "SBUX", "AMGN", "CHTR", "GILD", "CMCSA", "BKNG",
    "MDLZ", "FISV", "BIIB", "MU", "MCD", "AMAT", "ADP", "ILMN", "ATVI", "ISRG",
    "ADSK", "LRCX", "BIDU", "JD", "REGN", "WBA", "VRTX", "KHC", "WMT", "ZM", "MELI",
    "TMUS", "CTSH", "XLNX", "PCAR", "ALGN", "WDAY", "SIRI", "CTXS", "ADI", "EXC", "LULU",
    "MAR", "KLAC", "PAYX", "EA", "ILMN", "ALXN", "MNST", "BMRN", "EBAY", "CTAS", "VRSK",
    "IDXX", "CDNS", "NXPI", "ASML", "INCY", "KLAC", "MCHP", "SNPS", "SWKS", "VRSN",
    "WDC", "WYNN", "XLNX", "ZBRA", "ZTS", "AEP", "AIG", "ALL", "AXP", "BA", "BAC",
    "BK", "BLK", "C", "CAT", "CL", "COF", "COP", "COST", "CSCO", "CVS", "CVX",
    "DD", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD",
    "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI",
    "KO", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM",
    'BLND', 'BLNG', 'BLNKW', 'BLNGU', 'BLNGW', 'BLNK', 'BLPH', 'BLRX',
    'BLTE', 'BLU', 'BLUA', 'BLUE', 'BLW', 'BLX', 'BLZE', 'BMA', 'BMAC', 'BMAQ', 'BMAQR', 'BMAQU'
]

directory_path = '/share/us-stock-dataset/Data/Stocks'

# stocks_to_load = [s.split("\\")[1].replace(".csv", "") for s in glob.glob(directory_path + "//*.csv")]
#
# stock_number_to_load = 200
#
# stocks_to_load = stocks_to_load[1000:1000+stock_number_to_load]

dataloader = TimeseriesDataloader(directory_path, stocks_to_load, add_diff=False)

print(dataloader.get_data().shape)

Reading and merging CSV files: ['AAPL', 'TSLA', 'A', 'GOOG', 'AMZN', 'PYPL', 'NVDA', 'AMD', 'NFLX', 'MSFT', 'INTC', 'CSCO', 'ADBE', 'CRM', 'QCOM', 'TXN', 'AVGO', 'INTU', 'ORCL', 'COST', 'SBUX', 'AMGN', 'CHTR', 'GILD', 'CMCSA', 'BKNG', 'MDLZ', 'FISV', 'BIIB', 'MU', 'MCD', 'AMAT', 'ADP', 'ILMN', 'ATVI', 'ISRG', 'ADSK', 'LRCX', 'BIDU', 'JD', 'REGN', 'WBA', 'VRTX', 'KHC', 'WMT', 'ZM', 'MELI', 'TMUS', 'CTSH', 'XLNX', 'PCAR', 'ALGN', 'WDAY', 'SIRI', 'CTXS', 'ADI', 'EXC', 'LULU', 'MAR', 'KLAC', 'PAYX', 'EA', 'ILMN', 'ALXN', 'MNST', 'BMRN', 'EBAY', 'CTAS', 'VRSK', 'IDXX', 'CDNS', 'NXPI', 'ASML', 'INCY', 'KLAC', 'MCHP', 'SNPS', 'SWKS', 'VRSN', 'WDC', 'WYNN', 'XLNX', 'ZBRA', 'ZTS', 'AEP', 'AIG', 'ALL', 'AXP', 'BA', 'BAC', 'BK', 'BLK', 'C', 'CAT', 'CL', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD'

In [ ]:
device = 'cuda'

torch.manual_seed(1337)

config = TransformerConfig(
    input_embed=dataloader.n_channels(), 
    precision=torch.bfloat16, 
    batch_size=32, 
    block_size=16, 
    n_embed=128, 
    n_head=4, 
    n_layer=4, 
    learning_rate=0.001
)

runner = TransformerRunner(config, dataloader.get_data())

runner.train_iterate_n(25000)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(dataloader.token_codec.decode(runner.generate(context, max_new_tokens=2000)[0].tolist()))

1.629841 M parameters
step 0: train loss 207.5811, val loss 225.9600, time/iter 3.5762786865234374e-08
saved model version 1
step 100: train loss 185.3205, val loss 203.4738, time/iter 0.02938711881637573
step 200: train loss 102.8000, val loss 218.1405, time/iter 0.02815131425857544
step 300: train loss 97.1037, val loss 213.0945, time/iter 0.027819559574127198
step 400: train loss 90.5415, val loss 199.9008, time/iter 0.027685508728027344
step 500: train loss 81.8482, val loss 182.6332, time/iter 0.028083646297454835
step 600: train loss 75.5238, val loss 169.4792, time/iter 0.027992358207702638
step 700: train loss 73.0966, val loss 160.5869, time/iter 0.028052916526794435
step 800: train loss 70.0828, val loss 154.1905, time/iter 0.029692878723144533
step 900: train loss 67.5349, val loss 148.4275, time/iter 0.029540750980377197
step 1000: train loss 66.6387, val loss 145.3558, time/iter 0.029304401874542238
saved model version 2
step 1100: train loss 64.8395, val loss 142.6716, ti